In [ ]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD as TSVD
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict

import spacy
from spacy.lang.en.stop_words import STOP_WORDS as stopwords

In [ ]:
data = pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
train = data.iloc[:25000]
test = data.iloc[25000:]

In [ ]:
train.shape, test.shape

In [ ]:
train.head()

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
# Converting the text to lowercase

train['review'] = train['review'].apply(lambda x: str(x).lower())

In [ ]:
data.head()

### Contractions Expansion

In [ ]:
!pip install contractions

In [ ]:
import contractions

In [ ]:
contractions_dict = contractions.contractions_dict
contractions_dict

In [ ]:
def contraction_expansion(x):
    
    if type(x) is str:
        
        for key in contractions_dict:
            
            value = contractions_dict[key]
            
            x = x.replace(key, value)
            
        return x
    
    else:
        
        return x

In [ ]:
train['review'] = train['review'].apply(lambda x: contraction_expansion(x))

In [ ]:
train.head()

### Removing Emails

In [ ]:
def remove_emails(x):
    
    email_pattern = re.compile(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)")
    
    return re.sub(email_pattern, '', x)

In [ ]:
train['review'] = train['review'].apply(lambda x:remove_emails(x))

In [ ]:
train.sample(5)

### Removing HTML Tags

In [ ]:
train['review'] = train['review'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text().strip())

In [ ]:
train.iloc[6005][0]

In [ ]:
train.sample(5)

### Removing Special Characters

In [ ]:
def RemoveSpecialChars(x):
    
    x = re.sub(r'[^\w ]+', "", x)
    x = ' '.join(x.split())
    return x

In [ ]:
train['review'] = train['review'].apply(lambda x: RemoveSpecialChars(x))

In [ ]:
train.sample(5)

In [ ]:
train.iloc[6005][0]

### Lemmetization

In [ ]:
def lemme(x):
    
    x = str(x)
    x_list = []
    doc = nlp(x)
    
    for token in doc:
        lemma = token.lemma_
        
        if lemma == '-PRON-' or lemma == 'be':
            lemma = token.text
            
        x_list.append(lemma)
        
    return ' '.join(x_list)

In [ ]:
%%time
train['review'] = train['review'].apply(lambda x: lemme(x))

In [ ]:
train.sample(5)

### Tokenization using Text Blob

### Removing Stop Words

In [ ]:
stopwords

In [ ]:
len(stopwords)

In [ ]:
def RemoveStopWords(x):
    
    return ' '.join([word for word in x.split() if word not in stopwords])

In [ ]:
x = train.iloc[6005][0]

In [ ]:
# EXAMPLE CODE

print(x)
print()
print("length of x: ",len(x))

In [ ]:
x1 = RemoveStopWords(x)
x1

In [ ]:
len(x1)

In [ ]:
%%time

train['review'] = train['review'].apply(lambda x: RemoveStopWords(x))

In [ ]:
train.sample(5)

### Removing Rare Words

In [ ]:
text = ' '.join(train['review'])

In [ ]:
#text

In [ ]:
len(text)

In [ ]:
# Creating Frequency

text_series = pd.Series(text.split())

In [ ]:
freq_comm = text_series.value_counts()

In [ ]:
freq_comm

In [ ]:
rare_words = freq_comm[-82000:-1]
'rockumentarie' in rare_words

In [ ]:
rare_words

In [ ]:
# Removing 82000 rare occuring words 

train['review'] = train['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in rare_words]))

In [ ]:
train['review'].sample(5)

### Converting the Data into Vector

In [ ]:
train['sentiment'].value_counts()

In [ ]:
X = train['review']
y = train['sentiment']

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
X = tfidf.fit_transform(X)

In [ ]:
X.shape

In [ ]:
X

### Splitting Data into Training and Testing sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4, stratify = y)

In [ ]:
X_train.shape, X_test.shape

### Dimensionality reduction using Truncated Singular Value Decomposition

In [ ]:
%%time

#tsvd = TSVD(n_components=10000, random_state=4)
#X_train_tsvd = tsvd.fit_transform(X_train)

In [ ]:
#sum(tsvd.explained_variance_)

### Using SVC for Classification

In [ ]:
#clf_svc = SVC()

In [ ]:
%%time

#scores = cross_val_score(clf_svc, X_train, y_train, cv=6, n_jobs=-1)

In [ ]:
#scores

### Using Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf_lr = LogisticRegression()

In [ ]:
X_train

In [ ]:
%%time

scores = cross_val_score(clf_lr, X_train, y_train, cv=10, n_jobs=4)

In [ ]:
scores

In [ ]:
scores.mean()

In [ ]:
clf_lr.fit(X_train, y_train)

In [ ]:
y_test_pred = clf_lr.predict(X_test)

In [ ]:
print(classification_report(y_test, y_test_pred))

In [ ]:
confusion_matrix(y_test, y_test_pred)

In [ ]:
clf_lr.predict(tfidf.transform(['American Psycho deserved an Oscar, they were robbed']))

In [ ]:
y_real_pred = clf_lr.predict(tfidf.transform(test['review']))

In [ ]:
print(classification_report(test['sentiment'], y_real_pred))

In [ ]:
clf_lr.predict(tfidf.transform(["What hell was that, it's a masterpiece"]))